<a href="https://colab.research.google.com/github/danielle-l23/hello-world/blob/main/Medical_Home_Deliverable_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

__Group Members:__ 9079414778 (Matt) , 9071987730 (Caleb), 9058163032 (Danielle)

## Introduction 
Our analysis began with making corrections to our previous calculations of ‘The Medical Home’ indicator. We realized we had made a few oversights, the biggest one being that the binary indicators (for example, whether or not a child needs to see a specialist) were not included in the calculation of the final medical home predictor. As well, because we did not remove NAs from our calculation of the mean scores for ‘accessibility’, ‘family-centered’, ‘compassionate’, and ‘comprehensive’, this was resulting in many, many more NA values than was necessary. To correct these scores, we added an additional parameter of na.rm = TRUE for each of the rowMeans calculations for the 4 care categories; this allowed us to create a so-called “valid skip” (per our article) for those patients that did not have a relevant answer to a question about a specific health care need. We also adjusted our scoring, completely eliminating the binary indicators from our scoring system. From this, we received a much more reasonable number of children with a medical home and felt comfortable proceeding. 

__The commentary in this Deliverable will focus on what we did to create Table 4__

In [ ]:
library("epiDisplay")
library("questionr")
library('matrixStats')
library('dplyr')

ERROR: ignored

In [ ]:
load('H171.RData')

The left side of Table 4 examines the percentage of all children receiving guidance or screening from a doctor out of those receiving at least one doctor’s visit in that particular year. To start, we pulled out our variables necessary for building the table. To start, we need the variable that indicates whether a person has had a doctor’s visit in the past year. Because the article specified ‘office-based healthcare visit’, and the article focuses on doctors, we chose the variable ‘CHAPPT42’, which pulls data from the CAHPS section of MEPS, which focuses on ages 0 to 17 in the past 12 months. CHAPPT42 is an ordered factored variable, but we are still able to filter for all variables above 0 because those all correspond to positive values. We also took a second precaution to filter by less than or equal to age using the AGE14X variable, the numerical age variable.

In [ ]:
#Remove all people over 17yrs old and those with a missing value
H171_subset <- H171[which(H171$AGE14X < 18 & H171$AGE14X >= 0), ]


summary(H171_subset$AGE14X)

#Remove all missing values and people who have 0 office-based health care visits
H171_subset <- H171_subset[which(H171_subset$CHAPPT42 > 0), ]

summary(H171_subset$CHAPPT42)

Then, we subset the data to include only our variables of interest. Those include:  
* LAPBLT42, whether a doctor provided guidance about using a lapbelt and shoulder belt in the car   
* BOOST42, whether doctor has given advice about using a booster seat in the car   
* SAFEST42, whether doctor has given advice about using a safety seat in the car   
* HELMET42, whether the doctor has given advice about wearing a helmet while riding a bike   
* DENTAL42, whether doctor advised about a dental checkup   
* NOSMOK42, whether doctor has given advice about second-hand smoke  
* PHYSCL42, whether doctor has provided guidance about exercise or physical activity for the child   
* EATHLT42, whether doctor has provided guidance about eating healthy   
* MESBPR42, whether doctor has checked blood pressure of child   
* MESVIS42, whether doctor has ever checked child’s vision   
* MESWGT42, whether doctor has checked child’s weight   
* MESHGT42, whether doctor has checked child’s height 

All of our variables come from the ‘Child Preventative Care’ section of the MEPS data, as explained in the MEPS documentation.  

In [ ]:
table1_dat <- H171_subset[, c('HAVEUS42', 'PROVTY42', 'OFFHOU42', 'AFTHOU42', 
                              'PHNREG42', 'CHLIST42', 'CHPRTM42', 'TREATM42', 
                              'RESPCT42', 'DECIDE42', 'EXPLOP42', 'CHEXPL42', 
                              'CHSPEC42', 'CHEYRE42', 'CHILCR42', 'CHILWW42', 
                              'CHRTCR42', 'CHRTWW42', 'CHNDCR42', 'CHENEC42', 
                              'CHRESP42', "PLCTYP42")]

table2_dat <- H171_subset[,c('AGE14X', 'INSCOV14', 'sex', 'POVLEV14', 'racethx',
                             'RTHLTH42', 'REGION14')]

table4_dat <- H171_subset[,c('WHNLAP42', 'LAPBLT42', 'WHNBST42', 'PREVEN42',
                             'BOOST42', 'WHNSAF42', 'SAFEST42', 'WHNHEL42', 
                             'HELMET42', 'WHNDEN42', 'DENTAL42', 'WHNSMK42', 
                             'NOSMOK42', 'WHNPHY42', 'PHYSCL42', 'WHNEAT42', 
                             'EATHLT42', 'WHNBPR42', 'MESBPR42', 'MESVIS42', 
                             'WHNWGT42', 'MESWGT42', 'WHNHGT42', 'MESHGT42')]

## Table 2

In [ ]:
#Create new categorical variables to match categories in the paper for Table 2
table2_dat <- within(table2_dat, {
        Gender <- NA
        Gender[sex == 1] <- "Male"
        Gender[sex == 2] <- "Female"
        Age <- NA
        Age[AGE14X >= 0 & AGE14X <= 5] <- "0-5 Years"
        Age[AGE14X > 5 & AGE14X <= 11] <- "6-11 Years"
        Age[AGE14X > 11 & AGE14X <= 17] <- "12-17 Years"
        Race_ethnicity <- NA
        Race_ethnicity[racethx == 1] <- "Hispanic"
        Race_ethnicity[racethx == 2] <- "White, non-Hispanic"
        Race_ethnicity[racethx == 3] <- "Black, non-Hispanic"
        Race_ethnicity[racethx == 4 | racethx == 5] <- "Other, non-Hispanic"
        Insurance <- NA
        Insurance[INSCOV14 == 1] <- "Private"
        Insurance[INSCOV14 == 2] <- "Public"
        Insurance[INSCOV14 == 3] <- "Uninsured Full Year"
        FamilyIncome <- NA
        FamilyIncome[POVLEV14 < 100] <- "<100% FPL"
        FamilyIncome[POVLEV14 >= 100 & POVLEV14 < 200] <- "100%–199% FPL"
        FamilyIncome[POVLEV14 >= 200 & POVLEV14 < 400] <- "200%–399% FPL"
        FamilyIncome[POVLEV14 >= 400] <- ">=400% FPL"
        ChildHealth <- NA
        ChildHealth[RTHLTH42 == 1 | RTHLTH42 == 2] <- "Excellent/very good"
        ChildHealth[RTHLTH42 == 3] <- "Good"
        ChildHealth[RTHLTH42 == 4 | RTHLTH42 == 5] <- "Fair/poor"
        GeographicRegion <- NA
        GeographicRegion[REGION14 == 1] <- "Northeast"
        GeographicRegion[REGION14 == 2] <- "Midwest"
        GeographicRegion[REGION14 == 3] <- "South"
        GeographicRegion[REGION14 == 4] <- "West"
               })

In [ ]:
table2_dat$Age <- factor(table2_dat$Age,
                         levels = c("0-5 Years", "6-11 Years", "12-17 Years"))


table2_dat$Gender <- factor(table2_dat$Gender,
                            levels = c("Male", "Female"))


table2_dat$Race_ethnicity <- factor(table2_dat$Race_ethnicity, 
                                    levels = c("White, non-Hispanic", 
                                               "Black, non-Hispanic", 
                                               "Other, non-Hispanic", 
                                               "Hispanic"))


table2_dat$Insurance <- factor(table2_dat$Insurance,
                               levels = c("Private", "Public", "Uninsured Full Year"))


table2_dat$FamilyIncome <- factor(table2_dat$FamilyIncome,
                                  levels = c("<100% FPL", "100%–199% FPL", 
                                             "200%–399% FPL", ">=400% FPL"))


table2_dat$ChildHealth <- factor(table2_dat$ChildHealth,
                                 levels = c("Excellent/very good", "Good", 
                                            "Fair/poor"))


table2_dat$GeographicRegion <- factor(table2_dat$GeographicRegion,
                                      levels = c("Northeast", "Midwest", 
                                                 "South", "West"))

In [ ]:
summary(table2_dat)
str(table2_dat)

In [ ]:
#Age Breakdown
tab1(table2_dat$Age, graph = FALSE)

#Gender Breakdown
tab1(table2_dat$Gender, graph = FALSE)

#Race/ethnicity Breakdown
tab1(table2_dat$Race_ethnicity, graph = FALSE)

#Insurance Breakdown
tab1(table2_dat$Insurance, graph = FALSE)

#Family income Breakdown
tab1(table2_dat$FamilyIncome, graph = FALSE)

#Parent report HS Breakdown
tab1(table2_dat$ChildHealth,  graph = FALSE)

#Region Breakdown
tab1(table2_dat$GeographicRegion,  graph = FALSE)

## Table 1 / Table 3

Now, we need to recode the variables that are incorporated in to the calculation of the Medical Home designation. We've adjusted our binary indicators to now be coded as zero and one instead of carrying weight toward the Medical Home calculation. Also, the article mentioned that "Don't Know" responses were then coded as "No"s for the Medical Home calculation. In these instances, "Don't Know"s were originally coded as "-8". There is an exception to this for AFTHOU42 and OFFHOU42, regarding questions determining difficult accessing resources. It was assumed that if participants were answering "Don't Know", they likely did not have a difficult time. Because of it, these answers were coded as "Not at all difficult. 

In [ ]:
#USC
table1_dat <- within(table1_dat, {
      UsualSource <- NA
      UsualSource[HAVEUS42 == 1] <- "Yes"
      UsualSource[HAVEUS42 == 2 | HAVEUS42 == -8 | PLCTYP42 == 2] <- "No"
      ProvType <- NA
      ProvType[PROVTY42 == 1] <- "Facility"
      ProvType[PROVTY42 == 2 & 3] <- "Person"
      })

In [ ]:
#Accessible
#Variable for difficulty accessing care not included because it was removed from MEPS in 2014
table1_dat <- within(table1_dat, {
      Offhours <- NA
      Offhours[OFFHOU42 == 1] <- 100
      Offhours[OFFHOU42 == 2 | OFFHOU42 == -8] <- 0
      Diff_Aft <- NA
      Diff_Aft[AFTHOU42 == 1] <- 0
      Diff_Aft[AFTHOU42 == 2] <- 25
      Diff_Aft[AFTHOU42 == 3] <- 75
      Diff_Aft[AFTHOU42 == 4 | AFTHOU42 == -8] <- 100
      Phone <- NA
      Phone[PHNREG42 == 1] <- 0
      Phone[PHNREG42 == 2] <- 25
      Phone[PHNREG42 == 3] <- 75
      Phone[PHNREG42 == 4 | PHNREG42 == -8] <- 100
      })

In [ ]:
#Family
table1_dat <- within(table1_dat, {
      Listened <- NA
      Listened[CHLIST42 == 1] <- 0
      Listened[CHLIST42 == 2] <- 25
      Listened[CHLIST42 == 3] <- 75
      Listened[CHLIST42 == 4] <- 100
      EnoughTime <- NA
      EnoughTime[CHPRTM42 == 1] <- 0
      EnoughTime[CHPRTM42 == 2] <- 25
      EnoughTime[CHPRTM42 == 3] <- 75
      EnoughTime[CHPRTM42 == 4] <- 100
      Advice <- NA
      Advice[TREATM42 == 1] <- 100
      Advice[TREATM42 == 2 | TREATM42 == -8] <- 0
      Respect <- NA
      Respect[RESPCT42 == 1] <- 0
      Respect[RESPCT42 == 2] <- 25
      Respect[RESPCT42 == 3] <- 75
      Respect[RESPCT42 == 4] <- 100
      Decide <- NA
      Decide[DECIDE42 == 1] <- 0
      Decide[DECIDE42 == 2] <- 25
      Decide[DECIDE42 == 3] <- 75
      Decide[DECIDE42 == 4] <- 100
      ExplainOpt <- NA
      ExplainOpt[EXPLOP42 == 1] <- 100
      ExplainOpt[EXPLOP42 == 2 | EXPLOP42 == -8] <- 0
      Understand <- NA
      Understand[CHEXPL42 == 1] <- 0
      Understand[CHEXPL42 == 2] <- 25
      Understand[CHEXPL42 == 3] <- 75
      Understand[CHEXPL42 == 4] <- 100
      })

In [ ]:
#Comprehensive
table1_dat <- within(table1_dat, {
      Specialist <- NA
      Specialist[CHSPEC42 == 1] <- 1
      Specialist[CHSPEC42 == 2 | CHSPEC42 == -8] <- 0
      Access_Spec <- NA
      Access_Spec[CHEYRE42 == 1] <- 0
      Access_Spec[CHEYRE42 == 2] <- 50
      Access_Spec[CHEYRE42 == 3 | CHEYRE42 == 4 | CHEYRE42 == -8] <- 100
      Immediate <- NA
      Immediate[CHILCR42 == 1] <- 1
      Immediate[CHILCR42 == 2 | CHILCR42 == -8] <- 0
      CareNeeded <- NA
      CareNeeded[CHILWW42 == 1] <- 0
      CareNeeded[CHILWW42 == 2] <- 25
      CareNeeded[CHILWW42 == 3] <- 75
      CareNeeded[CHILWW42 == 4 | CHILWW42 == -8] <- 100
      Routine <- NA
      Routine[CHRTCR42 == 1] <- 1
      Routine[CHRTCR42 == 2 | CHRTCR42 == -8] <- 0
      RoutineCare <- NA
      RoutineCare[CHRTWW42 == 1] <- 0
      RoutineCare[CHRTWW42 == 2] <- 25
      RoutineCare[CHRTWW42 == 3] <- 75
      RoutineCare[CHRTWW42 == 4 | CHRTWW42 == -8] <- 100
      Believe <- NA
      Believe[CHNDCR42 == 1] <- 1
      Believe[CHNDCR42 == 2 | CHNDCR42 == -8] <- 0
      NecessaryCare <- NA
      NecessaryCare[CHENEC42 == 1] <- 0
      NecessaryCare[CHENEC42 == 2] <- 50
      NecessaryCare[ CHENEC42 == 3 |CHENEC42 == 4 | CHENEC42 == -8] <- 100
      })

In [ ]:
#Compass
table1_dat <- within(table1_dat, {
      RespPat <- NA
      RespPat[CHRESP42 == 1] <- 0
      RespPat[CHRESP42 == 2] <- 25
      RespPat[CHRESP42 == 3] <- 75
      RespPat[CHRESP42 == 4 | CHRESP42 == -8] <- 100
})

In [ ]:
#add to original working table for ease
table1_dat$access <- rowMeans(table1_dat[, c('Offhours', 'Diff_Aft', 'Phone')], na.rm = TRUE)

table1_dat$famcenter <- rowMeans(table1_dat[, c('Listened', 'EnoughTime','Advice',
                                                'Respect','Decide','ExplainOpt',
                                                'Understand')], na.rm = TRUE)

table1_dat$comprehensive <- rowMeans(table1_dat[, c('Access_Spec', 'CareNeeded',
                                                    'RoutineCare', 'NecessaryCare')], na.rm = TRUE)

In [ ]:
summary(table1_dat)

In [ ]:
#build the binary indicator for each category for medical homes
table1_dat <- within(table1_dat, {
  Accessible <- NA
  Accessible[access >= 75] <- 1
  Accessible[access < 75] <- 0
  })

table1_dat <- within(table1_dat, {
  FamilyCentered <- NA
  FamilyCentered[famcenter >= 75] <- 1
  FamilyCentered[famcenter < 75] <- 0
})

table1_dat <- within(table1_dat, {
  Comprehensive <- NA
  Comprehensive[comprehensive >= 75] <- 1
  Comprehensive[comprehensive < 75] <- 0
})

table1_dat <- within(table1_dat, {
  Compassionate <- NA
  Compassionate[RespPat >= 75] <- 1
  Compassionate[RespPat < 75] <- 0
})

table1_dat$MHScores <- rowSums(table1_dat[ , c('Accessible','FamilyCentered',
                                               'Comprehensive','Compassionate')])
table1_dat<- within(table1_dat, {
  MedicalHome <- NA
  MedicalHome[MHScores == 4] <- "Yes MH"
  MedicalHome[MHScores <= 3 | PLCTYP42 == 2] <- "No MH"
})

table(table1_dat$MedicalHome)

## Table 4 Percentages (Left Side)

In [ ]:
head(table4_dat)

dim(table4_dat)

Looking at the first few rows of the data, we can note all of the negative values, which upon looking at the codebook, we can understand as NA values. To make sure these are handled correctly, we’ll code all values that are not relevant to our analysis ("Not Ascertained, Don’t Know, Refused, and Inapplicable), as NA values via recoding and factoring each of our relevant variables.  

The variables are recoded by creating a completely new variable (given new names for easier interpretability), setting all the values to NA, then recoding the two values that are important to our analysis, which is a ‘No’ and ‘Yes’ response, which will be coded to 0 and 1, respectively.

In [ ]:
#Add in AGE14X variable to properly segment Booster, Safety, LapBelt
table4_dat$AGE14X <- H171_subset$AGE14X

#Create binary indicator variables for each health screening variable
table4_dat <- within(table4_dat, {
  LapBelt <- NA
  LapBelt[LAPBLT42 == 1 & WHNLAP42 == 1 & AGE14X > 9] <- 1
  LapBelt[LAPBLT42 == 2 & AGE14X > 9] <- 0
  Booster <- NA
  Booster[BOOST42 == 1 & WHNBST42 == 1 & AGE14X <= 9 & AGE14X >= 5] <- 1
  Booster[BOOST42 == 2 & AGE14X <= 9 & AGE14X >= 5] <- 0
  Safety <- NA
  Safety[SAFEST42 == 1 & WHNSAF42 == 1 & AGE14X <= 4] <- 1
  Safety[SAFEST42 == 2 & AGE14X <= 4]<- 0
  Helmet <- NA
  Helmet[HELMET42 == 1 & WHNHEL42 == 1] <- 1
  Helmet[HELMET42 == 2] <- 0
  Dental <- NA
  Dental[DENTAL42 == 1 & WHNDEN42 == 1] <- 1
  Dental[DENTAL42 == 2] <- 0
  NoSmoke <- NA
  NoSmoke[NOSMOK42 == 1 & WHNSMK42 == 1] <- 1
  NoSmoke[NOSMOK42 == 2] <- 0
  Exercise <- NA
  Exercise[PHYSCL42 == 1 & WHNPHY42 == 1] <- 1
  Exercise[PHYSCL42 == 2] <- 0
  Eat <- NA
  Eat[EATHLT42 == 1 & WHNEAT42 == 1]<- 1
  Eat[EATHLT42 == 2] <- 0
  BPR <- NA
  BPR[MESBPR42 == 1 &  WHNBPR42 == 1] <- 1
  BPR[MESBPR42 == 2] <- 0
  Vision <- NA
  Vision[MESVIS42 == 1] <- 1
  Vision[MESVIS42 == 2] <- 0
  Weight <- NA
  Weight[MESWGT42 == 1 &  WHNWGT42 == 1] <- 1
  Weight[MESWGT42 == 2] <- 0
  Height <- NA
  Height[MESHGT42 == 1 & WHNHGT42 == 1] <- 1
  Height[MESHGT42 == 2] <- 0
})

#Convert new variables to factor variables
table4_dat$LapBelt <- factor(table4_dat$LapBelt)
table4_dat$Booster <- factor(table4_dat$Booster)
table4_dat$Safety <-factor(table4_dat$Safety)
table4_dat$Helmet <- factor(table4_dat$Helmet)
table4_dat$Dental <- factor(table4_dat$Dental)
table4_dat$NoSmoke <- factor(table4_dat$NoSmoke)
table4_dat$Exercise <- factor(table4_dat$Exercise)
table4_dat$Eat <- factor(table4_dat$Eat)
table4_dat$BPR <- factor(table4_dat$BPR)
table4_dat$Vision <- factor(table4_dat$Vision)
table4_dat$Weight <- factor(table4_dat$Weight)
table4_dat$Height <- factor(table4_dat$Height)

The article also features two variables ‘Any Health Screening’ and ‘Any anticipatory guidance’. There was a lack of information on interpretation of these variables in the article - we chose to interpret them as if any of the screening variables in the ‘Receipt of a Health Screening’ section of the table were answered as ‘Yes’, ‘Any Health Screening would also be a ’yes’. We did this through an OR symbol within an ifelse statement, which applies a vectorized function to all rows of our dataframe. If the row meets our stipulations, it is set to ‘1’ in a new column, titled ‘AnyScreening’. If not, the value in ‘AnyScreening’ is ‘0’.

In [ ]:
#Create variable for "Any health screening" by checking if person received any of the health screenings 
table4_dat <- table4_dat %>%
  mutate(AnyScreening = case_when (
    table4_dat$Height == 1 | table4_dat$Weight == 1 | table4_dat$Vision == 1 | table4_dat$BPR == 1  ~ 1,
    TRUE ~ 0
  ))
table4_dat$AnyScreening <- factor(table4_dat$AnyScreening)

#Create variable for "Any anticipatory guidance" by checking if person received any of the guidance
table4_dat <- table4_dat %>%
  mutate(AnyGuidance = case_when (
    table4_dat$Eat == 1 | table4_dat$Exercise == 1 | table4_dat$NoSmoke == 1 | 
      table4_dat$Dental == 1 | table4_dat$Helmet == 1 | table4_dat$Safety == 1 |  
      table4_dat$Booster == 1 | table4_dat$LapBelt == 1  ~ 1,
    TRUE ~ 0
  ))
table4_dat$AnyGuidance <- factor(table4_dat$AnyGuidance)

We used the library ‘MatrixStats’ as a way to gather counts of an individual value across a set of columns. Matrixstats, as the name would imply, works only when the dataframe is converted into a matrix, so first we first format the data as a matrix. Then, because we want a percentage of total, one way to do it would be to take a total of all ‘Yes’ responses over a denominator of “Yes” responses + “No” responses. To start on that plan, we used the ‘colCounts’ function from MatrixStats to count each time a 1 occurred in each of the columns. The output was the count of 1s from each column, which we then saved to a dataframe.

In [ ]:
#Create data frame containing only the variables of interest
t4_var <- table4_dat[, c('Height', 'Weight', 'Vision', 'BPR', 'Eat', 'Exercise', 
                         'NoSmoke', 'Dental', 'Helmet', 'Safety', 'Booster', 
                         'LapBelt', 'AnyScreening', 'AnyGuidance')]

#Create matrix
H_matrix = as.matrix(t4_var)

#Display first 6 rows in matrix
head(H_matrix)


Then, we did the same for values of 0, saved it to a dataframe, and put both of our values as columns in this same dataframe. We added both columns to received a number of total applicable responses for each question. Then, we used that as the denominator in a new column that divides our 1 responses from the total responses for that question. This is our percentage of respondents that has received a treatment specified in a particular question. We mapped the names of the columns from our previous dataframe as new row values, so we can easily view which question corresponds with which percentage. 

In [ ]:
#Count the number of 1's (or Yes's) in the matrix for each variable (exclude NA's)
one_counts <- colCounts(H_matrix, value = 1, na.rm=TRUE)

#Create data frame from the count of ones for each variable
one_counts <- as.data.frame(one_counts)

#Count the number of 0's (or No's) in the matrix for each variable (exclude NA's)
zero_counts<- colCounts(H_matrix, value = 0, na.rm=TRUE)

#Create data frame from the count of zeroes for each variable
zero_counts <- as.data.frame(zero_counts)

#Append count of ones to count of zeroes data frame
zero_counts$one_counts <- one_counts$one_counts

#Create a new variable to capture the number of total respondents
zero_counts$total_respondents <- zero_counts$zero_counts + one_counts$one_counts

#Create new variable to capture the percentage of total respondents who received screening/guidance
zero_counts$percentage_pos <- zero_counts$one_counts / zero_counts$total_respondents

#Add variable names into zero_counts data frame
zero_counts$var_name <- names(t4_var)


We renamed the column to something more intuitive, and here are the results below. Notes on the results. It appears that screenings and guidance have increased in every single category of screening and anticipatory guidance. It seems that doctors appear to have gotten improved training on preventative medicine in children.

In [ ]:
#Rename zero_counts to a more understandable name
table4_dat_left <- zero_counts

#Display data frame with percentage of children receiving screening/guidance
table4_dat_left

## Table 4 Adjusted Odds Ratios

To be able to calculate our adjusted odds ratios for each variable we have to build a logistic regression using the required variables while changing the Table 4 variable used in each regression, then using the odds.ratio function we can calculate the adjusted odds ratio for each variable. 

The following variables were used for each of the respective covariates based on the original medical home models in the article.

AGE - AGE14X (continuous variable for age worked the best)
RACE/ETHNICITY (Recoded varaible 'Race_ethnicity')
GENDER (Recoded 'Gender')
INSURANCE STATUS (Recoded 'Insurance')
FAMLIY INCOME (Recoded 'FamilyIncome')
CHILD HEALTH STATUS (Recoded 'ChildHealth')
GEOGRAPHIC REGION (Recoded 'GeographicRegion')

Below we combined the data from all the previous tables, which we needed to be able to perform the regression. Then we go through each variable in table 4 to calculate the AOR for each variable. 


In [ ]:
#combine the data we created before
dat_comp<-cbind(table1_dat, table2_dat, table4_dat)
#create the table we need to calculate the log regressions for each variable
table4_log <- dat_comp[, c('Insurance','MedicalHome', 'AGE14X', 'Gender',  'Race_ethnicity',  
                           'FamilyIncome', 'ChildHealth', 'GeographicRegion', 
                           'Height', 'Weight', 'Vision', 'BPR', 'AnyScreening', 
                           'Eat', 'Exercise', 'NoSmoke', 'Dental', 'Helmet', 
                           'Safety', 'Booster', 'LapBelt', 'AnyGuidance', 'PREVEN42')]

table4_log$MedicalHome <- factor(table4_log$MedicalHome)

ERROR: ignored

In [ ]:

lr_height <- glm(MedicalHome ~ Height + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                   ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                 data = table4_log)
#AOR for height variable
odds.ratio(lr_height)


lr_weight<- glm(MedicalHome ~ Weight + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                data = table4_log)
#AOR for weight variable
odds.ratio(lr_weight)

lr_vision <- glm(MedicalHome ~ Vision + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                   ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                 data = table4_log)
#AOR for vision variable
odds.ratio(lr_vision)

lr_BPR <- glm(MedicalHome ~ BPR + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
               ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
              data = table4_log)
#AOR for blood pressure variable
odds.ratio(lr_BPR)

lr_screening <- glm(MedicalHome ~ AnyScreening + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                     ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), data = table4_log)
#AOR for any screening variable
odds.ratio(lr_screening)

In [ ]:
lr_diet <- glm(MedicalHome ~ Eat + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
               data = table4_log)
#AOR for diet variable
odds.ratio(lr_diet)

lr_exercise<- glm(MedicalHome ~ Exercise + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                  data = table4_log)
#AOR for exercise variable
odds.ratio(lr_exercise)

lr_smoke<- glm(MedicalHome ~ NoSmoke + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
               data = table4_log)
#AOR for smoking variable
odds.ratio(lr_smoke)

lr_dental<- glm(MedicalHome ~ Dental + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                data = table4_log)
#AOR for dental variable
odds.ratio(lr_dental)

lr_helmet<- glm(MedicalHome ~ Helmet + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                data = table4_log)
#AOR for helmet variable
odds.ratio(lr_helmet)

lr_safety<- glm(MedicalHome ~ Safety + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                data = table4_log)
#AOR for safety seat variable
odds.ratio(lr_safety)

lr_booster<- glm(MedicalHome ~ Booster + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                 data = table4_log)
#AOR for booster seat variable
odds.ratio(lr_booster)

lr_seatbelt<- glm(MedicalHome ~ LapBelt + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                  data = table4_log)
#AOR for seatbelt variable
odds.ratio(lr_seatbelt)

lr_guidance<- glm(MedicalHome ~ AnyGuidance + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), data = table4_log)
#AOR for anticipatory guidance variable
odds.ratio(lr_guidance)

## Table 4 Preventive Visit AOR

Below we are doing the same exact thing as the previous section, only we have subsetted the data by having a preventive visit, to get the final column of table 4 from the original article. 

In [ ]:
table4_plog <- table4_log[which(table4_log$PREVEN42 == 1), ]

summary(table4_plog)

In [ ]:
plr_height <- glm(MedicalHome ~ Height + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                   ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                 data = table4_plog)
#AOR for height variable - preventive visits subset
odds.ratio(plr_height)


plr_weight<- glm(MedicalHome ~ Weight + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                data = table4_plog)
#AOR for weight variable - preventive visits subset
odds.ratio(plr_weight)

plr_vision <- glm(MedicalHome ~ Vision + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                   ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                 data = table4_plog)
#AOR for vision variable - preventive visits subset
odds.ratio(plr_vision)

plr_BPR <- glm(MedicalHome ~ BPR + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
               ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
              data = table4_plog)
#AOR for blood pressure variable - preventive visits subset
odds.ratio(plr_BPR)

plr_screening <- glm(MedicalHome ~ AnyScreening + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                     ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), data = table4_plog)
#AOR for any screening variable - preventive visits subset
odds.ratio(plr_screening)

In [ ]:
plr_diet <- glm(MedicalHome ~ Eat + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
               data = table4_plog)
#AOR for diet variable - preventive visits subset
odds.ratio(plr_diet)

plr_exercise<- glm(MedicalHome ~ Exercise + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                  data = table4_plog)
#AOR for exercise variable - preventive visits subset
odds.ratio(plr_exercise)

plr_smoke<- glm(MedicalHome ~ NoSmoke + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
               data = table4_plog)
#AOR for smoking variable - preventive visits subset
odds.ratio(plr_smoke)

plr_dental<- glm(MedicalHome ~ Dental + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                data = table4_plog)
#AOR for dental variable - preventive visits subset
odds.ratio(plr_dental)

plr_helmet<- glm(MedicalHome ~ Helmet + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                data = table4_plog)
#AOR for helmet variable - preventive visits subset
odds.ratio(plr_helmet)

plr_safety<- glm(MedicalHome ~ Safety + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                data = table4_plog)
#AOR for safety seat variable - preventive visits subset
odds.ratio(plr_safety)

plr_booster<- glm(MedicalHome ~ Booster + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                 data = table4_plog)
#AOR for booster seat variable - preventive visits subset
odds.ratio(plr_booster)

plr_seatbelt<- glm(MedicalHome ~ LapBelt + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), 
                  data = table4_plog)
#AOR for seatbelt variable - preventive visits subset
odds.ratio(plr_seatbelt)

plr_guidance<- glm(MedicalHome ~ AnyGuidance + AGE14X + Gender + Race_ethnicity + FamilyIncome + 
                  ChildHealth + GeographicRegion + Insurance, family = binomial(link = "logit"), data = table4_plog)
#AOR for any guidance variable - preventive visits subset
odds.ratio(plr_guidance)